In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 

Data importing

In [15]:
df=pd.read_csv('train.csv')

Get data ready

In [85]:
def string_to_floats(string_list):
    letters = ' abcdefghijklmnopqrstuvwxyz,.-()/"'+"'0123456789"
    # Get the number of possible letter # add 1 for the ' ' character
    # Initialize an empty list to store the results
    result = []

    # Loop over each string in the input list
    for string in string_list:
        strng=str(string).lower()
        strng+=' '*25
        to_add=[]
        # for v in range(25):
            # if strng[v] in [',',',','.','-','(',')','/','"',"'"]:
            #     strng=strng[:v]+strng[v+1:]
            #     left+=1
        for v in range(25):
            to_add.append((letters.index(strng[v]))/len(letters))
        result.append(to_add)
    return np.array(result)

In [117]:
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9,10], [11, 12, 13, 14]]
print(tf.concat([t1,t2], axis=1))

features={}
answrs=np.array([])
for key in df.keys():
    print(str(key) +'   '+str(df[key].dtype))
    if key!= 'Survived':
        if df[key].dtype=='int64' and key!= 'PassengerId':  #ints to float32
            features[key]=np.array(df[key] - df[key].min()) / (df[key].max() - df[key].min())

        elif df[key].dtype=='object' and key in ['Sex', 'Embarked']:  #Sex and Embarked to float32
            to_add=np.array([])
            for one in df[key]:
                if one=='male':
                    to_add=np.append(to_add, 1)
                else:
                    to_add=np.append(to_add, 0)
            features[key]=to_add

        elif df[key].dtype=='object' and key in ['Name', 'Ticket', 'Cabin']:  #['Name', 'Ticket', 'Cabin'] to float32
            features[key]=tf.convert_to_tensor(string_to_floats(df[key]))

        elif df[key].dtype=='float64': #floats to float32
            features[key]=np.array(df[key] - df[key].min()) / (df[key].max() - df[key].min())

    else:
        answrs=np.append(answrs, df[key])
print(features.keys())
to_feed=[]
for key in features.keys():
    to_feed.append(tf.constant(features[key]))
to_feed=tf.concat(to_feed, axis=0, name='flattened_list')

tf.Tensor(
[[ 1  2  3  7  8  9 10]
 [ 4  5  6 11 12 13 14]], shape=(2, 7), dtype=int32)
PassengerId   int64
Survived   int64
Pclass   int64
Name   object
Sex   object
Age   float64
SibSp   int64
Parch   int64
Ticket   object
Fare   float64
Cabin   object
Embarked   object
dict_keys(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_10_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [891] vs. shape[1] = [891,25] [Op:ConcatV2] name: flattened_list

Modelka

In [89]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_dim=7),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(np.array(to_feed).T, answrs, epochs=10, batch_size=2)#, validation_data=(x_val, y_val))

C:\Users\alexb\AppData\Local\Temp\ipykernel_11952\1426083204.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(np.array(to_feed).T, answrs, epochs=10, batch_size=2)#, validation_data=(x_val, y_val))


ValueError: could not broadcast input array from shape (891,25) into shape (891,)

In [ ]:
model.save('Titanic')

INFO:tensorflow:Assets written to: Titanic\assets


INFO:tensorflow:Assets written to: Titanic\assets
